In [2]:
import pandas as pd

from pendulum import now

### Tabela 1612 - Área plantada, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras temporárias

In [3]:
dataset_name = 'sidra_tab1612_lavoura_temporaria'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

Área colhida (ha)

#### Bronze

In [ ]:
culturas = [2688, 2689, 2690, 2691, 2692, 2693, 2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707,
            2708, 2709, 2710, 2711, 2712, 2713, 2714, 2715, 2716, 109179, 109180]

contador = 1

ultimo_ano = now().year - 1


In [ ]:
for ano in range(2003, ultimo_ano):
    for cultura in culturas:
        if contador == 1:
            df_1612 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1612/n6/all/v/216/p/{ano}/c81/{cultura}')
            contador += 1
        else:
            df_1612 = pd.concat(
                [df_1612,
                 (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1612/n6/all/v/216/p/{ano}/c81/{cultura}')
                  .iloc[1:])],
                  ignore_index=True
                  )
            contador += 1

df_1612.to_json(f'{raiz}/bronze_{dataset_name}.json')

print(f'Foram contabilizadas {contador} repetições de dados de áreas de colheita raspados.')

#### Prata

| Código | Cultura                            |
|--------|------------------------------------|
| 2688   | Abacaxi*                           |
| 2689   | Algodão herbáceo (em caroço)       |
| 2690   | Alho                               |
| 2691   | Amendoim (em casca)                |
| 2692   | Arroz (em casca)                   |
| 2693   | Aveia (em grão)                    |
| 2694   | Batata-doce                        |
| 2695   | Batata-inglesa                     |
| 2696   | Cana-de-açúcar                     |
| 2697   | Cebola                             |
| 2698   | Centeio (em grão)                  |
| 2699   | Cevada (em grão)                   |
| 2700   | Ervilha (em grão)                  |
| 2701   | Fava (em grão)                     |
| 2702   | Feijão (em grão)                   |
| 2703   | Fumo (em folha)                    |
| 2704   | Juta (fibra)                       |
| 2705   | Linho (semente)                    |
| 2706   | Malva (fibra)                      |
| 2707   | Mamona (baga)                      |
| 2708   | Mandioca                           |
| 2709   | Melancia                           |
| 2710   | Melão                              |
| 2711   | Milho (em grão)                    |
| 2712   | Rami (fibra)                       |
| 2713   | Soja (em grão)                     |
| 2714   | Sorgo (em grão)                    |
| 2715   | Tomate                             |
| 2716   | Trigo (em grão)                    |
| 109179 | Girassol (em grão)                 |
| 109180 | Triticale (em grão)                |

In [ ]:
df_1612 = (pd.read_json(f'{raiz}/bronze_{dataset_name}.json')
          .drop(0, axis = 0)
          .filter(['D1C', 'D3C', 'D4C', 'D4N', 'V'])
          .assign(**{
              'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
              'hectares': lambda df: pd.to_numeric(df['V'], errors='coerce'),
              'municipio': lambda df: df['D1C'].astype(int),
              'ano': lambda df: df['D3C'].astype(int),
              'cd_cultura': lambda df: df['D4C'].astype(int),
              'nm_cultura': lambda df: df['D4N']
              })
          .convert_dtypes()
          .drop(['D1C', 'D3C', 'D4C', 'D4N', 'V'], axis='columns')
          .set_index(['uf', 'municipio', 'ano', 'cd_cultura', 'nm_cultura'])
          )

df_1612.to_parquet(f'{raiz}/prata_{dataset_name}.parquet')

### Ouro

In [59]:
dfs = ('df_839', 'df_1000', 'df_1001', 'df_1002')

tabelas = ('tab839_colheita_milho', 'tab1000_colheita_amendoim', 'tab1001_colheita_batata-inglesa', 'tab1002_colheita_feijao')

dfx = pd.DataFrame()

for df, tabela in zip(dfs, tabelas):
    globals()[df] = (pd.read_parquet(f'{raiz}/prata_sidra_{tabela}.parquet')
                     .reset_index()
                     .assign(hectares=lambda x: x.hectares.replace({0:1, pd.NA:1}))
                     )
    globals()[df + '_fps'] = (globals()[df]
        .filter(['uf', 'municipio', 'ano'])
        .drop_duplicates()
        .reset_index(drop=True)
        .assign(
            nm_cultura=globals()[df].nm_cultura,
            FPS=globals()[df].iloc[1::3, 5].reset_index(drop=True) / globals()[df].iloc[::3, 5].reset_index(drop=True) 
                    if (globals()[df].nm_cultura.iloc[0] == 'Milho (em grão)' or globals()[df].nm_cultura.iloc[0] == 'Amendoim (em casca)') else 
                    globals()[df].iloc[1::4, 5].reset_index(drop=True) / globals()[df].iloc[::4, 5].reset_index(drop=True)
        )
        .convert_dtypes()
    )
    dfx = pd.concat([dfx, globals()[df + '_fps']]).reset_index(drop=True)

In [62]:
df_1612 = pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')

In [63]:
df_1612_ps = (df_1612.reset_index()
              .merge(dfx,
                     how='left',
                     on=['uf', 'municipio', 'ano', 'nm_cultura']
                     )
              .assign(FPS=lambda x: x.FPS.replace({0:1, pd.NA:1}))
              .assign(valor=lambda df: df.hectares * df.FPS)
              .dropna(subset='valor')
              .drop(['hectares','FPS', 'nm_cultura'], axis='columns')
              .rename(columns={'cd_cultura': 'cultura'})
              .convert_dtypes()
              .assign(
                  safra='1a. safra',
                  valor=lambda df: df.apply(lambda x:
                                            x.valor * 0.17 if (x.cultura == 2689 and x.uf == 51) else
                                            x.valor * 0.79 if (x.cultura == 2689 and x.uf == 50) else
                                            0 if (x.cultura == 2689 and x.uf not in (17, 21, 22, 23, 24, 25, 26, 27, 28, 29, 50, 51)) else
                                            0 if (x.cultura in (2693, 2698, 2699, 2705, 2714, 2716, 109179, 109180)) else
                                            x.valor, axis = 1)
                                             )
              .reset_index(drop=True)
            )

In [64]:
df_1612_ss = (pd.merge(df_1612.dropna(subset='hectares'), df_1612_ps,
                      how='left',
                      left_on=['uf', 'municipio', 'ano', 'cd_cultura'],
                      right_on=['uf', 'municipio', 'ano', 'cultura']
                      )
                      .assign(valor=lambda df: df.apply(lambda x:
                                                        x.hectares - x.valor if pd.notna(x.valor) else x.hectares, axis = 1),
                              safra='2a. safra')
                      .drop('hectares', axis='columns')
                )

In [65]:
df_1612_final = (pd.concat([df_1612_ps, df_1612_ss], ignore_index=True)
                 .query('valor != 0')
                 .reset_index(drop=True)
                 .set_index(['uf', 'municipio', 'ano', 'cultura', 'safra'])
                 )

df_1612_final.to_csv(f'{raiz}/ouro_volume_agricola_lavouras_temporarias.csv')

In [71]:
df_1612_consolidado_munic = (df_1612_final.reset_index()
                             .groupby(['uf', 'municipio', 'ano', 'safra'])
                             .agg({'valor':sum})
                             )

df_1612_consolidado_munic.to_csv(f'{raiz}/ouro_volume_agricola_lavouras_temporarias_munic.csv')

In [73]:
df_1612_consolidado_uf = (df_1612_final.reset_index()
                          .groupby(['uf', 'ano', 'safra'])
                          .agg({'valor':sum})
                          )

df_1612_consolidado_uf.to_csv(f'{raiz}/ouro_volume_agricola_lavouras_temporarias_uf.csv')